In [1]:
import os
import numpy as np
import pandas as pd

import time
import requests

In [2]:
class SessionWithHeaderRedirection(requests.Session):
    AUTH_HOST = 'urs.earthdata.nasa.gov'
    def __init__(self, username, password):
        super().__init__()
        self.auth = (username, password)
    # Overrides from the library to keep headers when redirected to or from
    # the NASA auth host.
    def rebuild_auth(self, prepared_request, response):
        headers = prepared_request.headers
        url = prepared_request.url
        if 'Authorization' in headers:
            original_parsed = requests.utils.urlparse(response.request.url)
            redirect_parsed = requests.utils.urlparse(url)
            if (original_parsed.hostname != redirect_parsed.hostname) and \
                        redirect_parsed.hostname != self.AUTH_HOST and \
                        original_parsed.hostname != self.AUTH_HOST:
                del headers['Authorization']
        return

In [3]:
def download_my_urls(list_of_urls, out_dir, username, password):
    session = SessionWithHeaderRedirection(username, password)
    n_urls = len(list_of_urls)
    for iiu in range(n_urls):
        print(iiu)
        url = list_of_urls[iiu]
        path_and_name = os.path.join(out_dir, url[url.rfind('/') + 1:])
        try:
            # submit the request using the session
            response = session.get(url, stream=True)
            # print(response.status_code)
            # raise an exception in case of http errors
            response.raise_for_status()
            # save the file
            with open(path_and_name, 'wb') as fd:
                for chunk in response.iter_content(chunk_size=1024 * 1024):
                    fd.write(chunk)
        except requests.exceptions.HTTPError as e:
            # handle any errors here
            print(e)

In [8]:
os.environ['COMPUTERNAME'], os.environ['EARTH_DATA_USER']

KeyError: 'EARTH_DATA_USER'